In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [10]:
import nltk
from nltk.corpus import gutenberg

# Get the raw text of "Emma" by Jane Austen
text = gutenberg.raw("austen-emma.txt")

# Print the first 500 characters of the text
print(text[:500])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died t


In [11]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '6', '7', '8', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [12]:
vocab_size

77

In [13]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [14]:
x = 'Hello World!'

In [21]:
encoded = encode(x)

In [22]:
encoded

[29, 55, 62, 62, 65, 1, 44, 65, 68, 62, 54, 2]

In [23]:
decode(encoded)

'Hello World!'

In [24]:
data = torch.tensor(encode(text), dtype=torch.long)

In [25]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[58,  1, 34, 59, 69, 69,  1, 27],
        [65, 64, 70, 68, 51, 68, 75,  8],
        [70,  1, 72, 55, 68, 75,  1, 64],
        [62, 75,  0, 51, 64, 54,  1, 58]], device='cuda:0')
targets:
tensor([[ 1, 34, 59, 69, 69,  1, 27, 51],
        [64, 70, 68, 51, 68, 75,  8,  1],
        [ 1, 72, 55, 68, 75,  1, 64, 55],
        [75,  0, 51, 64, 54,  1, 58, 51]], device='cuda:0')
